CREATE SCHEMA Process;

In [ ]:
CREATE TABLE Process.WorkflowSteps (
    WorkFlowStepKey INT NOT NULL PRIMARY KEY,
    WorkFlowStepDescription NVARCHAR(100) NOT NULL,
    WorkFlowStepTableRowCount INT NULL DEFAULT 0,
    StartingDateTime DATETIME2(7) NULL DEFAULT SYSDATETIME(),
    EndingDateTime DATETIME2(7) NULL DEFAULT SYSDATETIME(),
    ClassTime CHAR(5) NULL DEFAULT '10:45' CHECK (ClassTime = '10:45'),
    UserAuthorizationKey INT NOT NULL
);

THE FOLLOWING IS THE Create a stored procedure Process.usp\_TrackWorkFlow to track each of the steps of your entire workflow of your project

In [ ]:

CREATE OR ALTER PROCEDURE [Process].[usp_TrackWorkFlows]
    @StartTime DATETIME2,
    @WorkFlowDescription NVARCHAR(100),
    @WorkFlowStepTableRowCount INT,
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    INSERT INTO Process.WorkflowSteps (
        WorkFlowStepKey,
        WorkFlowStepDescription,
        WorkFlowStepTableRowCount,
        StartingDateTime,
        EndingDateTime,
        ClassTime,
        UserAuthorizationKey
    )
    VALUES (
        (SELECT ISNULL(MAX(WorkFlowStepKey), 0) + 1 FROM Process.WorkflowSteps),
        @WorkFlowDescription,
        @WorkFlowStepTableRowCount,
        @StartTime,
        SYSDATETIME(),
        (SELECT ClassTime FROM DbSecurity.UserAuthorization WHERE UserAuthorizationKey = @UserAuthorizationKey),
        @UserAuthorizationKey
    );
END;

This is how we;d call the function

In [ ]:
DECLARE @StartTime DATETIME2 = SYSDATETIME();

/* Do your loading logic here */

EXEC Process.usp_TrackWorkFlows
    @StartTime = @StartTime,
    @WorkFlowDescription = 'Load DimProduct',
    @WorkFlowStepTableRowCount = (SELECT COUNT(*) FROM CH01-01-Dimension.DimProduct),
    @UserAuthorizationKey = 2;
